In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import os
from collections import Counter

from time import time

%matplotlib inline

import scipy.sparse

In [6]:
import urllib
import urllib2

In [127]:
def print_cat(cats):
    for cat in cats:
        print ', '.join(cat)

In [159]:
def extract_categories(dict_page):
    if 'categories' not in dict_page:
        return []

    categories = dict_page['categories']
    return [d['title'] for d in categories]

def encode_decode(v):
    # v = v.replace(' ', '_')
    if isinstance(v, unicode):
        return v.encode('utf8')
    else:
        return v

def request_categories(titles):
    url = 'http://ru.wikipedia.org/w/api.php'
    titles = [encode_decode(t) for t in titles]

    values = {'action': 'query',
              'prop': 'categories',
              'titles': '|'.join(titles),
              'format': 'json',
              'continue': ''}

    data = urllib.urlencode(values)
    req = urllib2.Request(url, data)
    response = urllib2.urlopen(req)
    resp_json = response.read()
    
    pages = json.loads(resp_json)['query']['pages'].values()

    cats = [extract_categories(p) for p in pages]
    return cats

##  Read processed MLP data

In [74]:
from collections import defaultdict

In [75]:
def id_counter(id_list):
    cnt = Counter()
    for el in id_list:
        cnt[el[u'element']] = el[u'count']
    return cnt

def rel_to_dict(rels):
    res = defaultdict(list)
    for r in rels:
        if not valid_def(r['definition']):
            continue
        res[r['identifier']].append((r['definition'], r['score']))
    return res

In [76]:
def_black_list = set()

def valid_def(definition):
    if len(definition) <= 3:
        return False

    return definition.lower() not in def_black_list

In [52]:
root = 'C:/tmp/mlp/ru-out/'

docs = []
titles = []
ids = []
rels = []

empty = 0
small = 0
uncategorized = 0

for f in os.listdir(root): 
    for line in file(root + f):
        doc = json.loads(line)

        title = doc['title']        

        if '(disambiguation)' in title:
            continue

        id_bag = id_counter(doc['identifiers'])
        if len(id_bag) <= 1:
            if len(id_bag) == 0:
                empty = empty + 1
            else:
                small = small + 1
            continue

        docs.append(doc)
        titles.append(title)
        ids.append(id_bag)

        id_rels = rel_to_dict(doc['relations'])
        rels.append(id_rels)

print empty, small, uncategorized

N_doc = len(ids)
print N_doc

8170 1244 0
5491


In [77]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(l), n):
        yield l[i:i+n]

In [174]:
wiki_categories = {}
second_try = []


In [ ]:
for title in titles:
    res = request_categories([title])[0]

    if res:
        wiki_categories[title] = res
    else:
        second_try.append(title)
    
    # print title, 'ready'

In [176]:
len(wiki_categories.keys())

2968

In [187]:
titles_try = set(titles) - set(wiki_categories.keys())
len(titles_try)

18

## Edit categories

In [226]:
def filter_cats(cats):
    return [c[10:] for c in cats if u'Категория:Википедия:' not in c]

In [197]:
# wiki_categories_o = wiki_categories

In [255]:
wiki_categories = {k: filter_cats(v) for (k, v) in wiki_categories_o.items()}

In [258]:
black_list = {u'Статьи со ссылками на Викисловарь', 
              u'Статьи, требующие уточнения источников',
              u'Физические величины по алфавиту', 
              u'Статьи, требующие проверки на грамматические и орфографические ошибки',
              u'Страницы, использующие повторяющиеся аргументы в вызовах шаблонов' }

In [259]:
def filter_blacklist(cats):
    return [c for c in cats if c not in black_list and not c.startswith(u'Незавершённые статьи')]

In [260]:
wiki_categories = {k: filter_blacklist(v) for (k, v) in wiki_categories.items()}

In [261]:
cnt = Counter()

for lst in wiki_categories.values():
    cnt.update(lst)

for (i, name) in cnt.most_common(50):
    print i, name

Функциональный анализ 104
Математический анализ 103
Комплексный анализ 101
Теория вероятностей 94
Теория чисел 86
Общая топология 81
Термодинамика 75
Теория графов 75
Дифференциальные уравнения 74
Квантовая механика 73
Линейная алгебра 70
Теория групп 66
Криптография 66
Физика твёрдого тела 62
Теория множеств 61
Дифференциальные уравнения в частных производных 60
Динамические системы 58
Химические элементы 57
Дифференциальная геометрия и топология 55
Оптика 54
Физические величины 54
Математическая логика 52
Многочлены 51
Физические законы и уравнения 51
Статистическая физика 49
Критерии подобия 48
Теоремы математического анализа 47
Дифференциальное исчисление 47
Астрофизика 46
Квантовая теория поля 46
Теоремы комплексного анализа 44
Теория категорий 43
Комбинаторика 42
Теоретическая механика 41
Магнетизм 41
Риманова (и псевдориманова) геометрия 41
Персоналии по алфавиту 41
Общая алгебра 39
Теория меры 39
Эконометрика 38
Интегральное исчисление 37
Алгоритмы 36
Кривые 36
Метрическая геом

In [267]:
singletons = set()

for (name, i) in cnt.most_common():
    if i == 1:
        singletons.add(name)

len(singletons)

1256

In [268]:
def filter_singletons(cats):
    return [c for c in cats if c not in singletons]

In [269]:
wiki_categories = {k: filter_singletons(v) for (k, v) in wiki_categories.items()}

In [271]:
sum(c for c in wiki_categories.items() if len(c) == 0)

0

In [284]:
import codecs

ru_cats_file = codecs.open('C:/tmp/mlp/ru_cats.txt', 'w', encoding='utf-8')

for k, v in wiki_categories.items():
    ru_cats_file.write('\t'.join([k] + list(v)))
    ru_cats_file.write('\n')

ru_cats_file.close()    

## Now let's find categories of these categories

In [273]:
all_cats = set()

for lst in wiki_categories.values():
    all_cats.update(lst)

len(all_cats)

1293

In [275]:
print list(all_cats)[0]

Соединения бора


In [277]:
print_cat(request_categories(['Категория:Соединения бора']))

Категория:Химические соединения по элементам


In [280]:
wiki_hierarchy = {}
second_try = []

In [283]:
for title in all_cats:
    res = request_categories([u'Категория:' + title])[0]

    if res:
        wiki_hierarchy[title] = res
    else:
        second_try.append(title)

In [287]:
wiki_hierarchy_c = {k: filter_cats(v) for (k, v) in wiki_hierarchy.items()}

In [290]:
for k, v in wiki_hierarchy_c.items()[:5]:
    print k, ':',
    print ', '.join(v)

Соединения бора : Химические соединения по элементам
Прикладная математика : Математика, Прикладные науки
Диофантовы уравнения : Теория чисел
Управляемый термоядерный синтез : Термоядерные реакции, Физика плазмы
Замечательные точки треугольника : Геометрия треугольника


In [291]:
ru_cats_file = codecs.open('C:/tmp/mlp/ru_cats_broader.txt', 'w', encoding='utf-8')

for k, v in wiki_hierarchy_c.items():
    ru_cats_file.write('\t'.join([k] + list(v)))
    ru_cats_file.write('\n')

ru_cats_file.close()    

In [293]:
np.mean([len(c) for c in wiki_categories.values()])

1.9822766307326878

In [292]:
broader_wiki_cats = {}

for k, v in wiki_categories.items():
    all_cats = set(v)
    
    for cat in v:
        if cat in wiki_hierarchy_c:
            all_cats.update(wiki_hierarchy_c[cat])
    
    broader_wiki_cats[k] = all_cats

In [294]:
np.mean([len(c) for c in broader_wiki_cats.values()])

5.2431938607710578

In [295]:
ru_cats_file = codecs.open('C:/tmp/mlp/ru_cats_extended.txt', 'w', encoding='utf-8')

for k, v in broader_wiki_cats.items():
    ru_cats_file.write('\t'.join([k] + list(v)))
    ru_cats_file.write('\n')

ru_cats_file.close()    

In [296]:
cnt = Counter()

for lst in broader_wiki_cats.values():
    cnt.update(lst)

for (i, name) in cnt.most_common(50):
    print i, name

Математический анализ 645
Математика 428
Физические науки 380
Общая алгебра 309
Разделы математики 276
Геометрия 274
Математические теоремы 240
Алгоритмы 225
Алгебра 223
Функции 216
Топология 214
Функциональный анализ 211
Прикладная математика 207
Дифференциальная геометрия и топология 200
Теория чисел 193
Физические величины 177
Скрытые категории 175
Квантовая физика 168
Физика 164
Механика 160
Математическая статистика 156
Математическая логика 156
Дискретная математика 150
Дифференциальные уравнения 142
Криптография 138
Общая топология 134
Комплексный анализ 133
Статистическая физика 131
Теория вероятностей и математическая статистика 131
Теория множеств 130
Линейная алгебра 130
Теория меры 130
Уравнения 127
Оптика 125
Физическая химия 124
Теория графов 123
Численные методы 120
Теория вероятностей 118
Физические законы и уравнения 118
Классическая механика 118
Термодинамика 117
Геометрические фигуры 116
Астрономия 111
Многообразия 108
Числа 104
Физика твёрдого тела 103
Физика конден